## Copyright (C) 2020 Sobhan Moradiyan Daghigh - All Rights Reserved
#### 12/24/2020
##    Implementation of ID3: This is not a complete implementation ... follow the ID3-Titanic.ipynb for complete implemen

In [1]:
import numpy as np
import pandas as pd

eps = np.finfo(float).eps

### Reading datasets 

In [57]:
train = pd.DataFrame(pd.read_csv('./datasets/train.csv'))

In [58]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,484,3,"Turkula, Mrs. (Hedwig)",female,63.0,0,0,4134.0,9.5875,NaN,S,1
1,276,1,"Andrews, Miss. Kornelia Theodosia",female,63.0,1,0,13502.0,77.9583,D7,S,1
2,223,3,"Green, Mr. George Henry",male,51.0,0,0,21440.0,8.0500,NaN,S,0
3,632,3,"Lundahl, Mr. Johan Svensson",male,51.0,0,0,347743.0,7.0542,NaN,S,0
4,26,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077.0,31.3875,NaN,S,1


In [59]:
train.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Survived
count,800.000000,800.000000,640.000000,800.000000,800.000000,5.890000e+02,800.000000,800.0000
mean,447.867500,2.311250,29.632031,0.541250,0.387500,2.649668e+05,32.592395,0.3850
std,258.858126,0.835312,14.617948,1.139044,0.811275,4.819212e+05,50.726982,0.4869
min,1.000000,1.000000,0.670000,0.000000,0.000000,6.950000e+02,0.000000,0.0000
25%,224.750000,2.000000,20.000000,0.000000,0.000000,2.416000e+04,7.925000,0.0000
50%,449.500000,3.000000,28.000000,0.000000,0.000000,2.361710e+05,14.500000,0.0000
75%,677.250000,3.000000,38.000000,1.000000,0.000000,3.477430e+05,31.275000,1.0000
max,891.000000,3.000000,80.000000,8.000000,6.000000,3.101298e+06,512.329200,1.0000


In [60]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  800 non-null    int64  
 1   Pclass       800 non-null    int64  
 2   Name         800 non-null    object 
 3   Sex          800 non-null    object 
 4   Age          640 non-null    float64
 5   SibSp        800 non-null    int64  
 6   Parch        800 non-null    int64  
 7   Ticket       589 non-null    float64
 8   Fare         800 non-null    float64
 9   Cabin        183 non-null    object 
 10  Embarked     800 non-null    object 
 11  Survived     800 non-null    int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 75.1+ KB


#### Therefore, it seems that (Age, Ticket, Cabin) columns have missing values.

### Ok now make a preprocess on attrs including define thresholds for non-categorical attrs.

#### Usefull attrs: { 'Pclass' - 'Sex' - 'Age' - 'Cabin' - 'Embarked' }

In [100]:
classes = train.groupby(by=['Sex', 'Pclass', 'Survived'])
classes.first()

PassengerId  \
Sex    Pclass Survived                
female 1      0                 499   
              1                 276   
       2      0                 855   
              1                 547   
       3      0                 793   
              1                 484   
male   1      0                 263   
              1                 840   
       2      0                 818   
              1                 544   
       3      0                 223   
              1                 763   

                                                                   Name   Age  \
Sex    Pclass Survived                                                          
female 1      0         Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  25.0   
              1                       Andrews, Miss. Kornelia Theodosia  63.0   
       2      0           Carter, Mrs. Ernest Courtenay (Lilian Hughes)  44.0   
              1                       Beane, Mrs. Edward (Ethel Clarke)  19.0   
       3      0                                 Sage, Miss. Stella Anna  10.0   
              1                                  Turkula, Mrs. (Hedwig)  63.0   
male   1      0                                       Taussig, Mr. Emil  52.0   
              1                                    Marechal, Mr. Pierre  35.0   
       2      0                                      Mallet, Mr. Albert  31.0   
              1                                       Beane, Mr. Edward  32.0   
       3      0                                 Green, Mr. George Henry  51.0   
              1                                   Barah, Mr. Hanna Assi  20.0   

                        SibSp  Parch    Ticket      Fare    Cabin Embarked  
Sex    Pclass Survived                                                      
female 1      0             1      2  113781.0  151.5500  C22 C26        S  
              1             1      0   13502.0   77.9583       D7        S  
       2      0             1      0  244252.0   26.0000      E77        S  
              1             1      0    2908.0   26.0000     E101        S  
       3      0             8      2  345773.0   69.5500       G6        S  
              1             0      0    4134.0    9.5875     E121        S  
male   1      0             1      1  110413.0   79.6500      E67        S  
              1             0      0   11774.0   29.7000      C47        C  
       2      0             1      1  233866.0   37.0042        D        C  
              1             1      0    2908.0   26.0000       F2        S  
       3      0             0      0   21440.0    8.0500      F38        S  
              1             0      0    2663.0    7.2292     E121        C

In [112]:
for grp, pdf in classes:
    print("Sex: ", grp[0], "   Pclass: ", grp[1], "    Survived: ", grp[2], "   count: ", len(pdf), "    null_counts: ", pdf['Age'].isnull().sum(), "    avg: ", np.mean(pdf['Age'].tolist().), "   value: ", pdf['Age'].tolist()[:15])

Sex:  female    Pclass:  1     Survived:  0    count:  3     null_counts:  0     avg:  25.666666666666668    value:  [25.0, 50.0, 2.0]
Sex:  female    Pclass:  1     Survived:  1    count:  82     null_counts:  9     avg:  nan    value:  [63.0, 56.0, 24.0, 24.0, nan, 54.0, 22.0, 50.0, 35.0, 48.0, 30.0, 51.0, 24.0, 60.0, 39.0]
Sex:  female    Pclass:  2     Survived:  0    count:  5     null_counts:  0     avg:  38.4    value:  [44.0, 57.0, 26.0, 27.0, 38.0]
Sex:  female    Pclass:  2     Survived:  1    count:  63     null_counts:  2     avg:  nan    value:  [19.0, 21.0, 24.0, 18.0, 29.0, 18.0, 28.0, 54.0, 42.0, 48.0, 41.0, 32.5, 34.0, 17.0, 29.0]
Sex:  female    Pclass:  3     Survived:  0    count:  63     null_counts:  15     avg:  nan    value:  [nan, nan, 10.0, nan, 20.0, 22.0, 30.0, 18.0, 21.0, 14.5, nan, nan, 30.5, 39.0, nan]
Sex:  female    Pclass:  3     Survived:  1    count:  65     null_counts:  23     avg:  nan    value:  [63.0, 38.0, nan, nan, 30.0, 14.0, 27.0, 26.0, 27.0

In [21]:
attrs = ['Pclass', 'Sex', 'Age', 'Embarked']

###  Summary of Survived lables.

In [22]:
train['Survived'].value_counts()

0    492
1    308
Name: Survived, dtype: int64

### Ok let's define a function for calculating Entropy

In [23]:
def get_entropy(dataset, attr):
    entropy = 0
    
    labels   = dataset['Survived'].unique()         # 0: dead, 1: alived
    features = dataset[attr].unique()              # Each feature for an attr. e.g. Sex attr has (Female, Male) features. 
    
    for feature in features:
        feature_entropy = 0
        
        for label in labels:
            cnt_of_each_grp = len(dataset[attr][dataset[attr]==feature][dataset['Survived']==label])
            cnt_of_all = len(dataset[attr][dataset[attr]==feature])
            fraction = np.divide(cnt_of_each_grp, np.add(cnt_of_all, eps))
            if fraction == 0:
                feature_entropy += 1   
            else: 
                feature_entropy += np.multiply(-fraction, np.log2(fraction))               
        
        fraction_of_features = np.divide(cnt_of_all, np.add(len(dataset), eps))
        entropy += np.multiply(fraction_of_features, feature_entropy)
        return entropy

### And another one for calculating root entropy 

In [24]:
def get_root_entropy(dataset):
    entropy = 0
    
    labels = dataset['Survived'].unique()         # 0: dead, 1: alived
    
    for label in labels:
        cnt_of_each_grp = dataset['Survived'].value_counts()[label]
        cnt_of_all = len(dataset['Survived'])
        fraction = np.divide(cnt_of_each_grp, np.add(cnt_of_all, eps))
        entropy += np.multiply(-fraction, np.log2(fraction))               
    
    return entropy

### And finaly a function for calculating Information Gain (IG)

In [25]:
def get_information_gain(dataset, attr):
    root_entropy = get_root_entropy(dataset)
    attr_entropy = get_entropy(dataset, attr)
    
    ig = root_entropy - attr_entropy
    
    return ig

In [26]:
get_root_entropy(train)

0.9614969508235551

In [27]:
entropy = {k:get_entropy(train, k) for k in attrs}
print("Entropy of each atrr: ")
entropy

Entropy of each atrr: 


{'Pclass': 0.4432098091637809,
 'Sex': 0.28643773973420256,
 'Age': 0.0025,
 'Embarked': 0.6734013070701766}

In [28]:
ig = {k:get_information_gain(train, k) for k in attrs}
print("IG of each atrr: ")
ig

IG of each atrr: 


{'Pclass': 0.5182871416597743,
 'Sex': 0.6750592110893525,
 'Age': 0.9589969508235552,
 'Embarked': 0.2880956437533785}

#### Therefore, till now 'Sex' attr is the best one

### Define a func which choose the largest information gain for each iterate.

In [29]:
def get_best_ig(ig):
    best = max(ig.values())
    attr_index = list(ig.values()).index(best)
    attr_name  = list(ig.keys())[attr_index]
    return attr_name

In [30]:
get_best_ig(ig)

'Age'

### Define a func which returns the subtree for each node.

In [31]:
def get_subtree_df(dataset, attr, feature):
    return pd.DataFrame(dataset[dataset[attr] == feature].reset_index(drop=True))

### Defind a func for checking purity of nodes.

In [32]:
def check_purity(dataset, subtree):
    alived = 0
    dead = 0
    
    for person in subtree['Survived']:
        if person == 0:
            dead += 1
        elif person == 1:
            alived += 1
    
    percentage = np.divide(max(alived, dead), np.add(alived, dead))
    if percentage > 0.9:
        return 1, [1 if alived > dead else 0]    # Pure
    elif percentage > 0:
        return 0, [1 if alived > dead else 0]    # Not pure
    else:
        return -1, _     # Theres not any instance of this feature in this subtree 

### And finally the main function for making the tree.

In [33]:
def iterate_tree(dataset, attrs, ig_list, subtree):

    node = get_best_ig(ig_list)
    ig_list.pop(node, None)
    
    features = dataset[node].unique()
    
    # Create a dictionary for our tree, just onece
    tree = {}
    tree[node] = {}
    
    for feature in features:
        sub = get_subtree_df(subtree, node, feature)
        purity, won_class = check_purity(dataset, sub)
            
        if purity == 1:
            tree[node][feature] = won_class    
            
        elif purity == 0:
            if not ig_list:
                tree[node][feature] = "fault" #won_class
            else:
                ig_lst = ig_list.copy()
                tree[node][feature] = iterate_tree(dataset, attrs, ig_lst, sub)
        else:
            continue
    return tree

In [34]:
subtree = iterate_tree(train, attrs, ig, train)
subtree

<ipython-input-32-305c0168837f>:11: RuntimeWarning: invalid value encountered in true_divide
  percentage = np.divide(max(alived, dead), np.add(alived, dead))
<ipython-input-32-305c0168837f>:11: RuntimeWarning: invalid value encountered in true_divide
  percentage = np.divide(max(alived, dead), np.add(alived, dead))
<ipython-input-32-305c0168837f>:11: RuntimeWarning: invalid value encountered in true_divide
  percentage = np.divide(max(alived, dead), np.add(alived, dead))
<ipython-input-32-305c0168837f>:11: RuntimeWarning: invalid value encountered in true_divide
  percentage = np.divide(max(alived, dead), np.add(alived, dead))
<ipython-input-32-305c0168837f>:11: RuntimeWarning: invalid value encountered in true_divide
  percentage = np.divide(max(alived, dead), np.add(alived, dead))
<ipython-input-32-305c0168837f>:11: RuntimeWarning: invalid value encountered in true_divide
  percentage = np.divide(max(alived, dead), np.add(alived, dead))
<ipython-input-32-305c0168837f>:11: RuntimeWar

{'Age': {63.0: [1],
  51.0: {'Sex': {'female': [1],
    'male': {'Pclass': {3: [0],
      1: {'Embarked': {'S': [1], 'C': [0]}},
      2: [0]}}}},
  38.0: {'Sex': {'female': {'Pclass': {3: [1], 1: [1], 2: [0]}},
    'male': {'Pclass': {3: [0], 1: {'Embarked': {'S': 'fault'}}}}}},
  34.0: {'Sex': {'female': [1], 'male': {'Pclass': {3: [0], 1: [1], 2: [0]}}}},
  20.0: {'Sex': {'female': [0],
    'male': {'Pclass': {3: {'Embarked': {'S': 'fault', 'C': 'fault'}}}}}},
  19.0: {'Sex': {'female': [1],
    'male': {'Pclass': {3: [0], 1: [0], 2: {'Embarked': {'S': 'fault'}}}}}},
  30.0: {'Sex': {'female': {'Pclass': {3: {'Embarked': {'S': 'fault'}},
      1: [1],
      2: [1]}},
    'male': [0]}},
  32.0: {'Sex': {'female': {'Pclass': {3: [0], 1: [1]}},
    'male': {'Pclass': {3: {'Embarked': {'S': 'fault', 'Q': [0]}},
      1: [1],
      2: {'Embarked': {'S': 'fault'}}}}}},
  9.0: {'Sex': {'female': [0],
    'male': {'Pclass': {3: {'Embarked': {'S': 'fault'}}}}}},
  21.0: {'Sex': {'female': {'

In [35]:
import pprint
pprint.pprint(subtree)

{'Age': {0.67: [1],
         0.75: [1],
         0.83: [1],
         0.92: [1],
         1.0: {'Sex': {'female': [1],
                       'male': {'Pclass': {2: [1],
                                           3: {'Embarked': {'S': 'fault'}}}}}},
         2.0: {'Sex': {'female': {'Pclass': {1: [0],
                                             2: [1],
                                             3: {'Embarked': {'S': 'fault'}}}},
                       'male': {'Pclass': {2: [1], 3: [0]}}}},
         3.0: {'Sex': {'female': {'Pclass': {2: [1], 3: [0]}}, 'male': [1]}},
         4.0: {'Sex': {'female': [1],
                       'male': {'Pclass': {3: {'Embarked': {'Q': [0],
                                                            'S': 'fault'}}}}}},
         5.0: [1],
         6.0: {'Sex': {'female': {'Pclass': {2: [1], 3: [0]}}, 'male': [1]}},
         7.0: {'Sex': {'female': [1], 'male': [0]}},
         8.0: {'Sex': {'female': [0], 'male': {'Pclass': {2: [1], 3: [0]}}}},
        